# Jorge Gut (5.6)

The flowchart below represents the processing of sugarcane for the production of molasses, brown sugar, crystal sugar, refined sugar, and confectioner's sugar.

The net profits per ton of each of these products are $30, $150, $170, $230, and $200, respectively. The minimum quantities to be produced weekly are 10, 25, 25, 90, and 40 tons, and the supply of sugarcane is limited to 10,000 tons per week. The processing steps are: (1) milling the sugarcane with a 50% yield, with the remainder being bagasse; (2) evaporating the juice with a 90% yield for syrup production; (3) cooking and crystallizing with a 30% yield for brown sugar and a 10% yield for molasses; (4) light refining of brown sugar with an 80% yield for the production of crystal sugar; (5) refining crystal sugar with a 70% yield for refined sugar and 25% for confectioner's sugar. The generated bagasse can be sold for a profit of $5 per ton.

a) Model the sugarcane processing based on the flowchart and yields.

b) Formulate an LP problem to maximize weekly profit.

c) Solve the problem using an optimization program.

## Problem Solving

Since it is a maximizing problem (our wish is to return the maximum sales possible), we can write the objective function as:

$$max. \,\, f(x)=\sum_{i \text{ = } product_1}^{product_n}c_ix_i$$

- $c$ is the cost
- $x$ is the products

Now, our eyes have to focus in declaring the constraints. Since we are maximizing the costs in terms of time, $x$ represents the products that we want to evaluate.

- Maximal amount: $x_{sugarcane} ≤ 10.000 \text{ ton/week}$
---
- Molasses: $x_{molasses} ≥ 10 \text{ ton/week}$
- Brown: $x_{brown} ≥ 25 \text{ ton/week}$
- Crystal: $x_{crystal} ≥ 25 \text{ ton/week}$
- Refined: $x_{refined} ≥ 90 \text{ ton/week}$
- Confectioner: $x_{confectioner} ≥ 40 \text{ ton/week}$
---
- 1: $0.5x_{bagasse} + 0.5x_{syrup} ≤ x_{sugarcane}$
- 2|3: $0.1x_{molasse} + 0.3x_{brown} ≤ 0.2x_{syrup}$
- 4: $0.8x_{crystal} ≤ 0.6 \cdot 0.2x_{syrup}$
- 5: $0.7x_{refined} + 0.25x_{confectioner} ≤ 0.2x_{crystal}$

Also, the variables are positive and real.

Finally, the problem definition is:
> $max.$ $f(x)=\sum_{i}^{n}c_ix_i$ <br>
$s.a:$ $\begin{cases}
        x_{sugarcane} ≤ 10000 \\
        x_{molasses} ≥ 10 \\
        x_{brown} ≥ 25 \\
        x_{crystal} ≥ 25 \\
        x_{refined} ≥ 90 \\
        x_{confectioner} ≥ 40 \\
        0.5x_{bagasse} + 0.5x_{syrup} ≤ x_{sugarcane} \\
        0.1x_{molasse} + 0.3x_{brown} ≤ 0.2x_{syrup} \\
        0.8x_{crystal} ≤ 0.6 \cdot 0.2x_{syrup} \\
        0.7x_{refined} + 0.25x_{confectioner} ≤ 0.2x_{crystal} \\
        x ≥ 0, \in \mathbb{R}^1
        \end{cases}$ <br>

## Pyomo

In [2]:
import platform
from pyomo.environ import *
# brew install glpk >> for MacOS


In [3]:
from pyomo.environ import *

# Model Declaration
model = ConcreteModel()

# Decision Variables
model.sugarcane = Var(domain=NonNegativeReals, initialize=10000)
model.syrup = Var(domain=NonNegativeReals, initialize=5000)
model.molasse = Var(domain=NonNegativeReals, initialize=100)
model.brown = Var(domain=NonNegativeReals)
model.crystal = Var(domain=NonNegativeReals)
model.refined = Var(domain=NonNegativeReals)
model.confection = Var(domain=NonNegativeReals)
model.bagasse = Var(domain=NonNegativeReals, initialize=5000)

# Objective Function Coefficients
model.costs = {
    'molasse': 30, 'brown': 150, 'crystal': 170,
    'refined': 230, 'confection': 200, 'bagasse': 5
}

# Objective Function
model.obj = Objective(
    expr=
    model.molasse*model.costs['molasse'] + model.brown*model.costs['brown'] + 
    model.crystal*model.costs['crystal'] + model.refined*model.costs['refined'] + 
    model.confection*model.costs['confection'] + model.bagasse*model.costs['bagasse'],
    sense=maximize
)

# Constraints
model.c1 = Constraint(expr=model.sugarcane <= 10000)  
model.c2 = Constraint(expr=model.molasse >= 10)  
model.c3 = Constraint(expr=model.brown >= 25) 
model.c4 = Constraint(expr=model.crystal >= 25) 
model.c6 = Constraint(expr=model.refined >= 90) 
model.c7 = Constraint(expr=model.confection >= 40) 
model.c8a = Constraint(expr=model.bagasse <= 0.5*model.sugarcane)
model.c8b = Constraint(expr=model.syrup <= 0.5*model.sugarcane)
model.c9a = Constraint(expr=model.brown <= 0.3*0.2*model.syrup)
model.c9b = Constraint(expr=model.molasse <= 0.1*0.2*model.syrup)
model.c10 = Constraint(expr=model.crystal <= 0.8*0.6*0.2*model.syrup)
model.c11a = Constraint(expr=model.refined <= 0.7*0.2*model.crystal)
model.c11b = Constraint(expr=model.confection <= 0.25*0.2*model.crystal)

# Solver
if platform.system() == "Darwin":  # For MacOS
    results = SolverFactory('glpk').solve(model)
elif platform.system() == "Windows":  # For Windows
    results = SolverFactory('glpk', executable=r'C:\glpk-4.55\w64\glpsol').solve(model)

# Display Results
if results.solver.termination_condition == TerminationCondition.optimal: 
    print('The solution is optimal.')
    print('========================')
    
    # Display Objective and Solution
    print(f'''Objective value: 
      f(x) = R$ {value(model.obj)}''')
    print(f'''Solution: 
      Molasse = {value(model.molasse)} 
      Brown = {value(model.brown)}
      Crystal = {value(model.crystal)}
      Refined = {value(model.refined)}
      Confection = {value(model.confection)}
      Bagasse = {value(model.bagasse)}
    ''')
else:
    print('No optimal solution found.')
    print(f"Solver status: {results.solver.termination_condition}")

No optimal solution found.
Solver status: other
